In [1]:
import os
import tarfile
import urllib
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import joblib
import mlflow
import mlflow.sklearn

Data preparation

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
fetch_housing_data()


def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)
housing=load_housing_data()

In [3]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [4]:

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]

        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)



num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])
housing_num = housing.drop("ocean_proximity", axis=1)
housing_num_tr = num_pipeline.fit_transform(housing_num)


num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [5]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
mlflow.autolog()

with mlflow.start_run() as run:
    svm_ = SVR(kernel='linear')
    svm_.fit(housing_prepared, housing_labels)
    housing_predictions = svm_.predict(housing_prepared)
    scores = cross_val_score(svm_, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
    svm_rmse_scores = np.sqrt(-scores)
    mlflow.log_metric("accuracy",scores.mean())
    mlflow.log_param("kernel",'linear')
    mlflow.sklearn.log_model(svm_,"model")
    display_scores(svm_rmse_scores) 

2021/04/11 22:04:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/tiger0436/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API."
2021/04/11 22:04:14 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Scores: [105342.09141998 112489.24624123 110092.35042753 113403.22892482
 110638.90119657 115675.8320024  110703.56887243 114476.89008206
 113756.17971227 111520.1120808 ]
Mean: 111809.84009600841
Standard deviation: 2762.393664321567


In [8]:
!mlflow ui

[2021-04-11 22:14:33 +0530] [1136] [INFO] Starting gunicorn 20.1.0
[2021-04-11 22:14:33 +0530] [1136] [INFO] Listening at: http://127.0.0.1:5000 (1136)
[2021-04-11 22:14:33 +0530] [1136] [INFO] Using worker: sync
[2021-04-11 22:14:33 +0530] [1139] [INFO] Booting worker with pid: 1139
^C

Aborted!
[2021-04-11 22:16:33 +0530] [1136] [INFO] Handling signal: int
[2021-04-11 22:16:33 +0530] [1139] [INFO] Worker exiting (pid: 1139)


In [9]:
with mlflow.start_run() as run:
    svm_ = SVR(kernel='rbf')
    svm_.fit(housing_prepared, housing_labels)
    housing_predictions = svm_.predict(housing_prepared)
    scores = cross_val_score(svm_, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
    svm_rmse_scores = np.sqrt(-scores)
    mlflow.log_metric("accuracy",scores.mean())
    mlflow.log_param("kernel",'rbf')
    mlflow.sklearn.log_model(svm_,"model")
    display_scores(svm_rmse_scores) 